In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
epochs = 50
batch_size = 128
noise_dim = 100
learning_rate = 0.0002
save_interval = 5

In [3]:
# ===================== DIRECTORIES =====================
os.makedirs("generated_samples", exist_ok=True)
os.makedirs("final_generated_images", exist_ok=True)


In [4]:
# ===================== LOAD DATASET (AUTO DOWNLOAD) =====================
# Fashion-MNIST dataset
(x_train, _), (_, _) = keras.datasets.fashion_mnist.load_data()


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Normalize to [-1, 1]
x_train = x_train.astype("float32")
x_train = (x_train - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(60000).batch(batch_size)

img_shape = (28, 28, 1)


In [6]:
# ===================== GENERATOR =====================
def build_generator():
    model = keras.Sequential([
        layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(noise_dim,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Reshape((7, 7, 256)),

        layers.Conv2DTranspose(128, 5, strides=1, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2DTranspose(64, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2DTranspose(1, 5, strides=2, padding="same",
                               use_bias=False, activation="tanh")
    ])
    return model


In [7]:
# ===================== DISCRIMINATOR =====================
def build_discriminator():
    model = keras.Sequential([
        layers.Conv2D(64, 5, strides=2, padding="same", input_shape=img_shape),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Conv2D(128, 5, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# ===================== LOSS & OPTIMIZERS =====================
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

gen_optimizer = keras.optimizers.Adam(learning_rate, beta_1=0.5)
disc_optimizer = keras.optimizers.Adam(learning_rate, beta_1=0.5)


In [9]:
# ===================== TRAINING STEP =====================
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [10]:
# ===================== SAVE GENERATED IMAGES =====================
def save_images(epoch):
    noise = tf.random.normal([25, noise_dim])
    generated_images = generator(noise, training=False)
    generated_images = (generated_images + 1) / 2.0

    fig = plt.figure(figsize=(5, 5))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap="gray")
        plt.axis("off")

    plt.savefig(f"generated_samples/epoch_{epoch:02d}.png")
    plt.close()


In [11]:
# ===================== TRAINING LOOP =====================
for epoch in range(1, epochs + 1):
    for image_batch in train_dataset:
        g_loss, d_loss = train_step(image_batch)

    print(f"Epoch {epoch}/{epochs} | D_loss: {d_loss:.4f} | G_loss: {g_loss:.4f}")

    if epoch % save_interval == 0:
        save_images(epoch)


Epoch 1/50 | D_loss: 1.4207 | G_loss: 0.6894
Epoch 2/50 | D_loss: 1.2922 | G_loss: 0.7431
Epoch 3/50 | D_loss: 1.3909 | G_loss: 0.8487
Epoch 4/50 | D_loss: 1.2793 | G_loss: 0.8140
Epoch 5/50 | D_loss: 1.2647 | G_loss: 0.8143
Epoch 6/50 | D_loss: 1.4574 | G_loss: 0.7142
Epoch 7/50 | D_loss: 1.3883 | G_loss: 0.7929
Epoch 8/50 | D_loss: 1.3112 | G_loss: 0.7790
Epoch 9/50 | D_loss: 1.2806 | G_loss: 0.7620
Epoch 10/50 | D_loss: 1.2935 | G_loss: 0.7626
Epoch 11/50 | D_loss: 1.3243 | G_loss: 0.7456
Epoch 12/50 | D_loss: 1.3551 | G_loss: 0.6743
Epoch 13/50 | D_loss: 1.3406 | G_loss: 0.6152
Epoch 14/50 | D_loss: 1.2822 | G_loss: 0.7858
Epoch 15/50 | D_loss: 1.3024 | G_loss: 0.7996
Epoch 16/50 | D_loss: 1.3525 | G_loss: 0.7287
Epoch 17/50 | D_loss: 1.3215 | G_loss: 0.7883
Epoch 18/50 | D_loss: 1.3074 | G_loss: 0.7367
Epoch 19/50 | D_loss: 1.3384 | G_loss: 0.8192
Epoch 20/50 | D_loss: 1.3045 | G_loss: 0.7696
Epoch 21/50 | D_loss: 1.3445 | G_loss: 0.8161
Epoch 22/50 | D_loss: 1.3273 | G_loss: 0.85

In [12]:
import shutil

# Path of the folder you want to download
folder_path = "/content/generated_samples"

# Zip the folder
shutil.make_archive("generated_samples_fm", "zip", folder_path)

# Download the zip file
from google.colab import files
files.download("generated_samples_fm.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>